In [2]:
import numpy as np 
import pandas as pd 
from data_processing import generate_vocab, process_data, create_dataloaders 
from model import get_pretrained_emb, EncoderDecoder, EncoderRNN, DecoderRNN, Attention, DecoderAttnRNN
from train_eval import train_and_eval, inspect_model, count_parameters, summarize_results, plot_single_learning_curve
import importlib
import pickle as pkl 

In [3]:
# parameters and hyperparameters 
SRC_LANG = 'vi'
TARG_LANG = 'en'
SRC_VOCAB_SIZE = 30000
TARG_VOCAB_SIZE = 30000
BATCH_SIZE = 32
SRC_MAX_SENTENCE_LEN = 10 
TARG_MAX_SENTENCE_LEN = 10
NUM_EPOCHS = 2000 
LR = 0.0005
NUM_LAYERS = 2 
ENC_HIDDEN_DIM = 300 
DEC_HIDDEN_DIM = 2 * ENC_HIDDEN_DIM

In [4]:
# # takes a long time to process, save to pickle for reimport in future 
# vocab = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)
# vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
# pkl.dump(vocab, open(vocab_filename, "wb"))

In [6]:
# reload from pickle 
vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
vocab = pkl.load(open(vocab_filename, "rb"))
data = process_data(SRC_LANG, TARG_LANG, vocab)
limited_data = process_data(SRC_LANG, TARG_LANG, vocab, sample_limit=BATCH_SIZE) 

In [7]:
# create dataloaders 
full_loaders = create_dataloaders(data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
fast_loaders = create_dataloaders(limited_data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)

In [10]:
# define model 

encoder = EncoderRNN(enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, src_max_sentence_len=SRC_MAX_SENTENCE_LEN,
                     pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))
# decoder = DecoderRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
#                      targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
#                      pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
decoder = DecoderAttnRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
                         targ_vocab_size=TARG_VOCAB_SIZE, src_max_sentence_len=SRC_MAX_SENTENCE_LEN, 
                         targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
                         pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
model = EncoderDecoder(encoder, decoder, vocab[TARG_LANG]['token2id']) 

In [11]:
train_and_eval(model=model, full_loaders=full_loaders, fast_loaders=fast_loaders, id2token=vocab[TARG_LANG]['id2token'],
               learning_rate=LR, num_epochs=NUM_EPOCHS, print_intermediate=True, save_checkpoint=True, model_name='test2', 
               lazy_eval=False, lazy_train=True, inspect=True)

Epoch: 0.00, Train Loss: 10.12, Val Loss: 10.24, Train BLEU: 0.72, Val BLEU: 0.70
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: excess of of of of of of of of behaving of of of of of the the the mother-in-law a of of of of the the the businesses of of of of of of the the the the the the the the the the the
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: uphill the the the the the the the the iconic of of of of of of of of the the the the the the the the the falling of of of of the the the the the the

Epoch: 9.00, Train Loss: 5.59, Val Loss: 8.42, Train BLEU: 0.58, Val BLEU: 0.22
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: of of of of of of of of of of of of of of of of of of the the the the the of of of of of of of of of of of of of the the the the the the the the the
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: the the the the the of of of of of of of of of of of of of the the the the the the the the the of of of of of of of of of the the of of of of of of of
Epoch: 10.00, Train Loss: 5.29,

Epoch: 18.00, Train Loss: 4.37, Val Loss: 9.24, Train BLEU: 2.17, Val BLEU: 0.32
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: rachel of of of of of of of of like the , , , , , , , to the it it it it it it it like the , , , , , , , to the it it it it it it it
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to the the it it it it it it that the , , , , , , , to the it it it it it it it like the , , , , , , , to the it it it it it it it
Epoch: 19.00, Train Loss: 4.33, Val Loss: 9.36, Train BLEU: 2.54, 

Epoch: 27.00, Train Loss: 4.08, Val Loss: 10.03, Train BLEU: 5.70, Val BLEU: 0.30
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: rachel when francisco euphore euphore euphore euphore euphore euphore like the the the the the the the the to to the the the the the the the like the the the the , , , the to to the the the the the the the
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to to the the the the the the the like the the the the , , , , to to the the the the the the the like the the the the the t

Epoch: 35.00, Train Loss: 3.83, Val Loss: 10.43, Train BLEU: 4.17, Val BLEU: 0.23
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: rachel when francisco is over over over over over like the the of of of of of of to the the the the the the the the like the the of of of of of of to to the the the the the the the
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to to the the the the the the the like the the of of of of of of to to the the the the the the the like the the the of of of of the to to the the th

...

Epoch: 1968.00, Train Loss: 0.00, Val Loss: 15.23, Train BLEU: 10.36, Val BLEU: 1.11
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind are francisco of over over over over over : the the varied molecules and chamber models world to to almost and weight blow smog our looking : have big varied molecules and chamber world world to to community and weight blow smog our the
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to to thousand and the because smog have the like have big varied small it at and 

Epoch: 1976.00, Train Loss: 0.00, Val Loss: 15.24, Train BLEU: 10.36, Val BLEU: 1.11
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind are francisco of over over over over over : the the varied molecules and chamber models world to to almost and weight blow smog our looking : have big varied molecules and chamber world world to to community and weight blow smog our the
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to to thousand and the because smog have the like have big varied small it at and 

Epoch: 1984.00, Train Loss: 0.00, Val Loss: 15.24, Train BLEU: 10.36, Val BLEU: 1.12
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind are francisco of over over over over over : the the varied molecules and chamber models world to to almost and weight blow smog our looking : have big varied molecules and chamber world world to to community and weight blow smog our the
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to to thousand and the because smog have the like have big varied small it at and 

Epoch: 1992.00, Train Loss: 0.00, Val Loss: 15.25, Train BLEU: 10.36, Val BLEU: 1.11
Inspecting model on training data...
REFERENCE TRANSLATION: rachel pike : the science behind a climate headline in 4 minutes , atmospheric chemist rachel pike provides i &apos;d like to talk to you today about headlines that look like this when they have to they are both two branches of the same field
MODEL TRANSLATION: behind are francisco of over over over over over : the the varied molecules and chamber models world to they almost and weight blow smog our looking : have big varied molecules and chamber world world to to community and weight blow smog our the
Inspecting model on validation data...
REFERENCE TRANSLATION: when i was little , i thought my country and i was very proud . in school , we spent a lot of time although i often wondered about the outside world , when i was seven years old , i saw
MODEL TRANSLATION: to to thousand and the because smog have the like have big varied small it at an

[{'epoch': 0.0,
  'train_bleu': 0.7210203638464378,
  'train_loss': 10.12065315246582,
  'val_bleu': 0.7041990396651131,
  'val_loss': 10.239965438842773},
 {'epoch': 1.0,
  'train_bleu': 0.6915281505062035,
  'train_loss': 9.849146842956543,
  'val_bleu': 0.22474076683759026,
  'val_loss': 10.133549690246582},
 {'epoch': 2.0,
  'train_bleu': 0.6438642310573189,
  'train_loss': 9.415060997009277,
  'val_bleu': 0.21944904089555517,
  'val_loss': 9.946893692016602},
 {'epoch': 3.0,
  'train_bleu': 0.5841289077087483,
  'train_loss': 8.799692153930664,
  'val_bleu': 0.21944904089555517,
  'val_loss': 9.674530982971191},
 {'epoch': 4.0,
  'train_bleu': 0.5734063902120405,
  'train_loss': 8.12705135345459,
  'val_bleu': 0.21944904089555517,
  'val_loss': 9.37114143371582},
 {'epoch': 5.0,
  'train_bleu': 0.56781151253584,
  'train_loss': 7.502461910247803,
  'val_bleu': 0.21944904089555517,
  'val_loss': 9.089516639709473},
 {'epoch': 6.0,
  'train_bleu': 0.5734063902120405,
  'train_loss':